# Day 2: Environment Setup & API Basics

## GenAI Foundation Training Program

### Learning Objectives

By the end of this notebook, you will be able to:

1. Set up and authenticate with OpenAI, Anthropic (Claude), and Google AI (Gemini) APIs
2. Make basic API calls to all three providers
3. Generate **structured JSON outputs** using Pydantic models
4. Implement **error handling** patterns for production use
5. Compare providers and understand their differences

### Prerequisites

- Basic Python knowledge
- Google Colab account (you're already here!)
- API keys from:
  - OpenAI (https://platform.openai.com/api-keys)
  - Anthropic (https://console.anthropic.com/settings/keys)
  - Google AI (https://aistudio.google.com/app/apikey)


---

## Section 1: Introduction & Setup Overview

### Google Colab vs Local Development

**Google Colab** provides a cloud-based Jupyter notebook environment with:
- ✅ Pre-installed Python 3.10+
- ✅ Free GPU/TPU access
- ✅ No local setup required
- ✅ Easy collaboration and sharing
- ✅ Pre-configured environment

**For local development**, you would typically:
1. Install Python 3.10 or higher
2. Create a virtual environment:
   ```bash
   python -m venv venv
   source venv/bin/activate  # On Windows: venv\Scripts\activate
   ```
3. Install packages with pip

**For this training, we'll use Google Colab exclusively** - everything is already set up!

---

## Section 2: Package Installation

Let's install all the required packages for working with the three major LLM providers.

**Packages:**
- `openai` - OpenAI's official Python SDK
- `anthropic` - Anthropic's official Python SDK for Claude
- `google-generativeai` - Google's Gemini API SDK
- `pydantic` - Data validation and structured outputs

In [ ]:
# Install all required packages
!pip install -q openai anthropic google-generativeai pydantic

print("✅ All packages installed successfully!")

---

## Section 3: Import All Required Libraries

Let's import everything we'll need for this notebook.

In [ ]:
# Core imports
import json
import time
from typing import List, Optional, Literal, Any, Dict
from enum import Enum

# LLM Provider SDKs
from openai import OpenAI
from anthropic import Anthropic
import google.generativeai as genai

# Pydantic for structured outputs
from pydantic import BaseModel, Field, ValidationError

# Google Colab specific
from google.colab import userdata

print("✅ All imports successful!")

---

## Section 4: API Key Management & Security

### 🔒 Security Best Practices

**NEVER hardcode API keys in your code!** Instead:
- ✅ Use environment variables
- ✅ Use secrets management (Google Colab Secrets)
- ✅ Rotate keys regularly
- ✅ Set spending limits
- ❌ Never commit keys to Git
- ❌ Never log keys in console

### Setting Up Google Colab Secrets

1. Click the **🔑 (key icon)** in the left sidebar
2. Add these three secrets:
   - `OPENAI_API_KEY` - Your OpenAI API key
   - `ANTHROPIC_API_KEY` - Your Anthropic API key
   - `GOOGLE_API_KEY` - Your Google AI API key
3. Toggle **"Notebook access"** ON for each key

### Cost Monitoring

- OpenAI: https://platform.openai.com/usage
- Anthropic: https://console.anthropic.com/settings/usage
- Google AI: https://aistudio.google.com/app/usage

In [ ]:
# API Key Validation Helper
def validate_api_keys():
    """Validate that all required API keys are present in Colab secrets."""
    required_keys = ['OPENAI_API_KEY', 'ANTHROPIC_API_KEY', 'GOOGLE_API_KEY']
    missing_keys = []
    
    for key in required_keys:
        try:
            value = userdata.get(key)
            if not value or len(value) < 10:
                missing_keys.append(key)
        except Exception:
            missing_keys.append(key)
    
    if missing_keys:
        print("❌ Missing or invalid API keys:")
        for key in missing_keys:
            print(f"   - {key}")
        print("\n⚠️  Please add these keys in Google Colab Secrets (🔑 icon in sidebar)")
        return False
    else:
        print("✅ All API keys validated successfully!")
        return True

# Validate keys
validate_api_keys()

In [ ]:
# Retrieve API keys from Colab secrets
try:
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    ANTHROPIC_API_KEY = userdata.get('ANTHROPIC_API_KEY')
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    print("✅ API keys loaded successfully!")
except Exception as e:
    print(f"❌ Error loading API keys: {e}")
    print("Please set up your API keys in Google Colab Secrets.")

---

## Section 5: OpenAI API Setup & First Calls

### About OpenAI

- **Models:** GPT-4o, GPT-4o-mini, GPT-4 Turbo
- **Strengths:** Best-in-class reasoning, wide adoption, excellent documentation
- **Use cases:** General-purpose tasks, coding, analysis, creative writing
- **Pricing:** https://openai.com/api/pricing/

In [ ]:
# Initialize OpenAI client
openai_client = OpenAI(api_key=OPENAI_API_KEY)

print("✅ OpenAI client initialized!")

In [ ]:
# Simple text completion example
def simple_openai_call(prompt: str, model: str = "gpt-4o-mini"):
    """Make a simple OpenAI API call."""
    response = openai_client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a helpful AI assistant."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7,
        max_tokens=500
    )
    
    return {
        "content": response.choices[0].message.content,
        "model": response.model,
        "usage": {
            "prompt_tokens": response.usage.prompt_tokens,
            "completion_tokens": response.usage.completion_tokens,
            "total_tokens": response.usage.total_tokens
        }
    }

# Test it!
result = simple_openai_call("Explain what a Large Language Model is in 2 sentences.")
print("📝 Response:", result['content'])
print("\n📊 Tokens used:", result['usage'])

### Structured Outputs with Pydantic (OpenAI)

**Why structured outputs?**
- Guarantees valid JSON response
- Type safety and validation
- Easier integration with downstream systems
- Better error handling

OpenAI supports **native Pydantic integration** with `response_format`.

In [ ]:
# Define Pydantic model for product reviews
class SentimentEnum(str, Enum):
    POSITIVE = "positive"
    NEGATIVE = "negative"
    NEUTRAL = "neutral"
    MIXED = "mixed"

class ProductReview(BaseModel):
    """Structured product review analysis."""
    sentiment: SentimentEnum = Field(description="Overall sentiment of the review")
    confidence_score: float = Field(ge=0.0, le=1.0, description="Confidence in sentiment (0-1)")
    key_points: List[str] = Field(description="Main points mentioned in review")
    pros: List[str] = Field(default_factory=list, description="Positive aspects")
    cons: List[str] = Field(default_factory=list, description="Negative aspects")
    recommended: bool = Field(description="Whether product is recommended")

print("✅ ProductReview model defined!")

In [ ]:
# Use structured output with OpenAI
def analyze_review_openai(review_text: str) -> ProductReview:
    """Analyze product review using OpenAI with structured output."""
    response = openai_client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a product review analyzer. Extract structured insights from reviews."},
            {"role": "user", "content": f"Analyze this product review:\n\n{review_text}"}
        ],
        response_format=ProductReview
    )
    
    return response.choices[0].message.parsed

# Test with a sample review
sample_review = """
I've been using this laptop for 3 months now. The battery life is absolutely incredible - 
easily 12+ hours of real work. The display is stunning with great color accuracy. 
However, the trackpad is way too sensitive and causes accidental clicks. 
The keyboard is also a bit shallow for my taste. Despite these minor issues, 
I'd definitely recommend this for students and professionals who need portability.
"""

analysis = analyze_review_openai(sample_review)
print("📊 Structured Analysis:\n")
print(json.dumps(analysis.model_dump(), indent=2))

---

## Section 6: Anthropic (Claude) API Setup & First Calls

### About Anthropic Claude

- **Models:** Claude Opus 4.5, Claude Sonnet 4, Claude Haiku 4.5
- **Strengths:** Excellent instruction following, strong reasoning, safety-focused
- **Use cases:** Analysis, research, creative tasks, coding
- **Pricing:**
  - Claude Haiku 4.5: $0.80/1M input tokens, $4.00/1M output tokens
  - Claude Sonnet 4: $3.00/1M input tokens, $15.00/1M output tokens
  - Claude Opus 4.5: $15.00/1M input tokens, $75.00/1M output tokens

In [ ]:
# Initialize Anthropic client
anthropic_client = Anthropic(api_key=ANTHROPIC_API_KEY)

print("✅ Anthropic client initialized!")

In [ ]:
# Simple message example with Claude
def simple_anthropic_call(prompt: str, model: str = "claude-haiku-4-20251015"):
    """Make a simple Anthropic API call."""
    response = anthropic_client.messages.create(
        model=model,
        max_tokens=1024,
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    
    return {
        "content": response.content[0].text,
        "model": response.model,
        "usage": {
            "input_tokens": response.usage.input_tokens,
            "output_tokens": response.usage.output_tokens
        }
    }

# Test it!
result = simple_anthropic_call("Explain what a transformer architecture is in 2 sentences.")
print("📝 Response:", result['content'])
print("\n📊 Tokens used:", result['usage'])

### Structured Outputs with Pydantic (Anthropic)

Anthropic uses **tool calling** to generate structured JSON outputs. We define a "tool" with our Pydantic schema.

In [ ]:
# Define Pydantic model for meeting notes
class ActionItem(BaseModel):
    task: str = Field(description="The action item to be completed")
    assignee: str = Field(description="Person responsible")
    deadline: Optional[str] = Field(description="Due date if mentioned")
    priority: str = Field(default="medium", description="Priority level: low, medium, high")

class MeetingNotes(BaseModel):
    """Structured meeting notes."""
    meeting_title: str
    date: str
    participants: List[str]
    key_decisions: List[str] = Field(description="Important decisions made")
    action_items: List[ActionItem]
    next_meeting: Optional[str] = Field(description="Next meeting date")
    summary: str = Field(description="Brief meeting summary")

print("✅ MeetingNotes model defined!")

In [ ]:
# Use structured output with Anthropic (tool calling pattern)
def extract_meeting_notes_anthropic(meeting_text: str) -> MeetingNotes:
    """Extract structured meeting notes using Claude with tool calling."""
    response = anthropic_client.messages.create(
        model="claude-sonnet-4-20250514",
        max_tokens=2048,
        tools=[
            {
                "name": "extract_meeting_notes",
                "description": "Extract structured meeting notes from meeting transcript",
                "input_schema": MeetingNotes.model_json_schema()
            }
        ],
        messages=[
            {
                "role": "user",
                "content": f"Extract structured meeting notes from this transcript:\n\n{meeting_text}"
            }
        ]
    )
    
    # Extract tool use from response
    for block in response.content:
        if block.type == "tool_use":
            return MeetingNotes(**block.input)
    
    raise ValueError("No tool use found in response")

# Test with sample meeting transcript
sample_meeting = """
Product Roadmap Meeting - January 15, 2025

Attendees: Sarah (PM), John (Engineering), Lisa (Design), Mike (Marketing)

Key Decisions:
- We will prioritize the mobile app redesign for Q1
- Backend API v2 will be postponed to Q2
- Marketing budget increased by 20% for new feature launch

Action Items:
- Sarah to finalize mobile app wireframes by Jan 22
- John to provide API migration estimate by Jan 20
- Lisa to create design system documentation (high priority) - deadline Feb 1
- Mike to schedule user research sessions next week

Next meeting scheduled for January 29, 2025 at 2 PM.
"""

notes = extract_meeting_notes_anthropic(sample_meeting)
print("📋 Structured Meeting Notes:\n")
print(json.dumps(notes.model_dump(), indent=2))

---

## Section 7: Google AI (Gemini) API Setup & First Calls

### About Google AI (Gemini)

- **Models:** Gemini 3 Pro, Gemini 2.5 Pro, Gemini 2.5 Flash, Gemini 2.0 Flash
- **Strengths:** Multimodal (text, images, video), long context windows, cost-effective
- **Use cases:** Content generation, multimodal analysis, summarization
- **Pricing:**
  - Gemini 2.0 Flash: $0.075/1M input tokens, $0.30/1M output tokens (estimated)
  - Gemini 2.5 Flash: $0.10/1M input tokens, $0.40/1M output tokens (estimated)
  - Gemini 3 Pro: $2.50/1M input tokens, $10.00/1M output tokens (estimated)

In [ ]:
# Configure Google AI
genai.configure(api_key=GOOGLE_API_KEY)

print("✅ Google AI configured!")

In [ ]:
# Simple content generation with Gemini
def simple_google_call(prompt: str, model_name: str = "gemini-2.0-flash-exp"):
    """Make a simple Google AI API call."""
    model = genai.GenerativeModel(model_name)
    response = model.generate_content(prompt)
    
    return {
        "content": response.text,
        "model": model_name
    }

# Test it!
result = simple_google_call("Explain what attention mechanism is in neural networks in 2 sentences.")
print("📝 Response:", result['content'])
print("\n📊 Model:", result['model'])

### Structured Outputs with Pydantic (Google AI)

Google AI supports **JSON mode** with schema definition using `response_mime_type` and `response_schema`.

In [ ]:
# Define Pydantic model for recipe analysis
class RecipeAnalysis(BaseModel):
    """Structured recipe analysis."""
    cuisine_type: str = Field(description="Type of cuisine (e.g., Italian, Indian, Mexican)")
    difficulty_level: Literal["easy", "medium", "hard"] = Field(description="Cooking difficulty")
    estimated_time_minutes: int = Field(description="Total cooking time in minutes")
    ingredients_count: int = Field(description="Number of ingredients required")
    dietary_tags: List[str] = Field(description="Dietary classifications (vegetarian, vegan, gluten-free, etc.)")
    main_protein: Optional[str] = Field(description="Primary protein source if any")
    summary: str = Field(max_length=200, description="Brief recipe summary")

print("✅ RecipeAnalysis model defined!")

In [ ]:
# Use structured output with Google AI
def analyze_recipe_google(recipe_text: str) -> RecipeAnalysis:
    """Analyze recipe using Google AI with structured output."""
    model = genai.GenerativeModel(
        'gemini-2.0-flash-exp',
        generation_config={
            "response_mime_type": "application/json",
            "response_schema": RecipeAnalysis
        }
    )
    
    response = model.generate_content(
        f"Analyze this recipe and provide structured information:\n\n{recipe_text}"
    )
    
    return RecipeAnalysis.model_validate_json(response.text)

# Test with sample recipe
sample_recipe = """
Chickpea Curry

Ingredients:
- 2 cans chickpeas
- 1 can coconut milk
- 2 tomatoes, diced
- 1 onion, chopped
- 3 cloves garlic
- 2 tbsp curry powder
- 1 tsp cumin
- Salt and pepper
- Fresh cilantro
- Rice for serving

Instructions:
1. Sauté onion and garlic until fragrant (5 minutes)
2. Add curry powder and cumin, cook 1 minute
3. Add tomatoes and cook until softened (10 minutes)
4. Add chickpeas and coconut milk, simmer 15 minutes
5. Season with salt and pepper
6. Garnish with cilantro and serve over rice

Total time: 35 minutes
"""

recipe_info = analyze_recipe_google(sample_recipe)
print("🍳 Structured Recipe Analysis:\n")
print(json.dumps(recipe_info.model_dump(), indent=2))

---

## Section 8: Error Handling Patterns

### Common API Errors

**Understanding HTTP Status Codes:**
- `401 Unauthorized` - Invalid or missing API key
- `429 Too Many Requests` - Rate limit exceeded
- `400 Bad Request` - Invalid request parameters
- `500 Internal Server Error` - Provider-side issues
- `503 Service Unavailable` - Temporary service outage
- Network timeouts - Connection issues

### Why Error Handling Matters

- **Production reliability** - Graceful degradation
- **User experience** - Better error messages
- **Debugging** - Easier troubleshooting
- **Cost control** - Prevent runaway costs

In [ ]:
# Import provider-specific exceptions
from openai import OpenAIError, RateLimitError, APIError, AuthenticationError
from anthropic import APIError as AnthropicAPIError

# Basic error handling for OpenAI
def safe_openai_call(prompt: str, model: str = "gpt-4o-mini"):
    """OpenAI call with comprehensive error handling."""
    try:
        response = openai_client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7,
            max_tokens=500
        )
        return {
            "success": True,
            "data": response.choices[0].message.content,
            "usage": response.usage.model_dump()
        }
    
    except AuthenticationError as e:
        return {
            "success": False,
            "error": "authentication_error",
            "message": "Invalid API key. Please check your credentials.",
            "details": str(e)
        }
    
    except RateLimitError as e:
        return {
            "success": False,
            "error": "rate_limit",
            "message": "Rate limit exceeded. Please try again later.",
            "details": str(e)
        }
    
    except APIError as e:
        return {
            "success": False,
            "error": "api_error",
            "message": "OpenAI API error occurred.",
            "details": str(e)
        }
    
    except Exception as e:
        return {
            "success": False,
            "error": "unknown",
            "message": "Unexpected error occurred.",
            "details": str(e)
        }

# Test error handling
result = safe_openai_call("What is machine learning?")
print("Result:", json.dumps(result, indent=2))

In [ ]:
# Error handling for Anthropic
def safe_anthropic_call(prompt: str, model: str = "claude-haiku-4-20251015"):
    """Anthropic call with comprehensive error handling."""
    try:
        response = anthropic_client.messages.create(
            model=model,
            max_tokens=1024,
            messages=[{"role": "user", "content": prompt}]
        )
        return {
            "success": True,
            "data": response.content[0].text,
            "usage": {
                "input_tokens": response.usage.input_tokens,
                "output_tokens": response.usage.output_tokens
            }
        }
    
    except AnthropicAPIError as e:
        return {
            "success": False,
            "error": "api_error",
            "message": "Anthropic API error occurred.",
            "details": str(e)
        }
    
    except Exception as e:
        return {
            "success": False,
            "error": "unknown",
            "message": "Unexpected error occurred.",
            "details": str(e)
        }

# Test error handling
result = safe_anthropic_call("What is deep learning?")
print("Result:", json.dumps(result, indent=2))

---

## Section 9: Comparative Examples

Let's compare all three providers on the **same task** to understand their differences in:
- Response quality
- Token usage
- Response time
- Cost

In [ ]:
# Email classification - unified model for all providers
class EmailClassification(BaseModel):
    """Structured email classification."""
    category: Literal["sales", "support", "inquiry", "complaint", "spam"]
    urgency: Literal["low", "medium", "high", "critical"]
    sentiment: Literal["positive", "neutral", "negative"]
    requires_response: bool
    suggested_department: str
    key_topics: List[str] = Field(max_items=3)
    summary: str = Field(max_length=200)

# Sample email for testing
test_email = """
Subject: URGENT: System Down - Payment Processing Failed

Hi Support Team,

Our payment processing system has been down for the past 2 hours. 
We've lost over $50,000 in potential sales. Multiple customers are 
complaining about failed transactions. This is completely unacceptable!

We need this fixed immediately. Our business is suffering.

Please escalate to your engineering team ASAP.

John Smith
CEO, TechStartup Inc.
"""

print("📧 Test Email Loaded")

In [ ]:
# Compare all three providers

def classify_email_openai(email_text: str) -> dict:
    """Classify email using OpenAI."""
    start = time.time()
    
    response = openai_client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are an email classification assistant."},
            {"role": "user", "content": f"Classify this email:\n\n{email_text}"}
        ],
        response_format=EmailClassification
    )
    
    elapsed = time.time() - start
    
    return {
        "provider": "OpenAI GPT-4o-mini",
        "result": response.choices[0].message.parsed.model_dump(),
        "response_time": f"{elapsed:.2f}s",
        "tokens": response.usage.total_tokens,
        "cost_estimate": f"${(response.usage.prompt_tokens * 0.15 + response.usage.completion_tokens * 0.60) / 1_000_000:.6f}"
    }

def classify_email_anthropic(email_text: str) -> dict:
    """Classify email using Anthropic."""
    start = time.time()
    
    response = anthropic_client.messages.create(
        model="claude-haiku-4-20251015",
        max_tokens=1024,
        tools=[{
            "name": "classify_email",
            "description": "Classify an email into structured categories",
            "input_schema": EmailClassification.model_json_schema()
        }],
        messages=[{"role": "user", "content": f"Classify this email:\n\n{email_text}"}]
    )
    
    elapsed = time.time() - start
    
    # Extract tool use
    result = None
    for block in response.content:
        if block.type == "tool_use":
            result = EmailClassification(**block.input).model_dump()
    
    total_tokens = response.usage.input_tokens + response.usage.output_tokens
    
    return {
        "provider": "Anthropic Claude Haiku 4.5",
        "result": result,
        "response_time": f"{elapsed:.2f}s",
        "tokens": total_tokens,
        "cost_estimate": f"${(response.usage.input_tokens * 0.80 + response.usage.output_tokens * 4.00) / 1_000_000:.6f}"
    }

def classify_email_google(email_text: str) -> dict:
    """Classify email using Google AI."""
    start = time.time()
    
    model = genai.GenerativeModel(
        'gemini-2.0-flash-exp',
        generation_config={
            "response_mime_type": "application/json",
            "response_schema": EmailClassification
        }
    )
    
    response = model.generate_content(f"Classify this email:\n\n{email_text}")
    elapsed = time.time() - start
    
    result = EmailClassification.model_validate_json(response.text)
    
    return {
        "provider": "Google Gemini 2.0 Flash",
        "result": result.model_dump(),
        "response_time": f"{elapsed:.2f}s",
        "tokens": "N/A",
        "cost_estimate": "~$0.000010 (estimated)"
    }

# Run comparison
print("🔄 Running comparison across all three providers...\n")

openai_result = classify_email_openai(test_email)
anthropic_result = classify_email_anthropic(test_email)
google_result = classify_email_google(test_email)

# Display results
for result in [openai_result, anthropic_result, google_result]:
    print(f"\n{'='*60}")
    print(f"Provider: {result['provider']}")
    print(f"Response Time: {result['response_time']}")
    print(f"Tokens: {result['tokens']}")
    print(f"Cost: {result['cost_estimate']}")
    print(f"\nClassification:")
    print(json.dumps(result['result'], indent=2))

---

## Section 10: Best Practices & Production Tips

### Token Management

**Cost optimization strategies:**
- Use cheaper models for simple tasks (gpt-4o-mini, claude-haiku, gemini-flash)
- Count tokens before sending to avoid surprises
- Cache responses when possible
- Use streaming for long responses
- Set `max_tokens` to prevent runaway costs

### Security Checklist

✅ **DO:**
- Store API keys in secrets/environment variables
- Rotate keys regularly (every 90 days)
- Monitor usage dashboards daily
- Set spending limits
- Sanitize user inputs
- Log errors but not sensitive data

❌ **DON'T:**
- Hardcode API keys
- Commit keys to version control
- Share keys in chat/email
- Log API keys
- Ignore rate limits
- Skip error handling

### Production Considerations

When building production applications:
- Always implement error handling
- Monitor API usage and costs
- Use appropriate models for each task
- Validate all inputs and outputs
- Keep API keys secure
- Test with real-world data

---

## Section 11: Hands-On Exercises

Now it's your turn! Complete these exercises to practice what you've learned.

### Exercise 1: Multi-Provider Sentiment Analyzer

**Task:** Create a sentiment analyzer that:
1. Takes a customer review as input
2. Analyzes it using all three providers (OpenAI, Anthropic, Google)
3. Returns structured sentiment analysis from each
4. Compares the results

**Requirements:**
- Use Pydantic model for structured output
- Include error handling
- Compare response times

In [ ]:
# Exercise 1: Your code here

class SentimentAnalysis(BaseModel):
    """Sentiment analysis result."""
    sentiment: Literal["positive", "negative", "neutral"]
    confidence: float = Field(ge=0, le=1)
    key_phrases: List[str]
    recommendation: str

# TODO: Implement multi_provider_sentiment_analysis function
def multi_provider_sentiment_analysis(review_text: str) -> Dict[str, Any]:
    """
    Analyze sentiment using all three providers and compare results.
    
    Args:
        review_text: Customer review to analyze
    
    Returns:
        Dictionary with results from all providers
    """
    # Your implementation here
    pass

# Test your implementation
test_review = """
The new smartphone is amazing! Camera quality is top-notch and battery lasts all day.
However, it's quite expensive and the charger is sold separately which is disappointing.
Overall, worth it if you can afford the premium price.
"""

# Uncomment to test:
# results = multi_provider_sentiment_analysis(test_review)
# print(json.dumps(results, indent=2))

### Exercise 2: Email Parser with Error Handling

**Task:** Build an email parser that:
1. Extracts structured data from raw email text
2. Implements comprehensive error handling
3. Falls back to another provider if the first fails

**Requirements:**
- Pydantic model for email structure
- Try OpenAI first, fallback to Anthropic on error
- Handle validation errors gracefully

In [ ]:
# Exercise 2: Your code here

class EmailData(BaseModel):
    """Structured email data."""
    sender: str
    subject: str
    urgency: Literal["low", "medium", "high"]
    category: str
    action_required: bool
    deadline: Optional[str]
    summary: str

# TODO: Implement robust_email_parser function
def robust_email_parser(email_text: str) -> EmailData:
    """
    Parse email with error handling and fallback.
    
    Args:
        email_text: Raw email text
    
    Returns:
        Structured email data
    """
    # Your implementation here
    pass

# Test your implementation
test_email_ex2 = """
From: jane.doe@company.com
Subject: Q1 Budget Approval Needed by Friday

Hi Team,

We need to finalize the Q1 budget by end of day Friday.
Please review the attached spreadsheet and send your approvals.

This is blocking our vendor negotiations, so please prioritize.

Thanks,
Jane
"""

# Uncomment to test:
# result = robust_email_parser(test_email_ex2)
# print(json.dumps(result.model_dump(), indent=2))

### Exercise 3: Provider Comparison Dashboard

**Task:** Create a comparison function that:
1. Takes a prompt and runs it through all three providers
2. Measures response time for each
3. Estimates cost for each
4. Returns a comparison summary

**Requirements:**
- Handle errors for each provider separately
- Track timing accurately
- Calculate cost estimates

In [ ]:
# Exercise 3: Your code here

# TODO: Implement compare_all_providers function
def compare_all_providers(prompt: str) -> Dict[str, Any]:
    """
    Run the same prompt through all providers and compare results.
    
    Args:
        prompt: Text prompt to send to all providers
    
    Returns:
        Comparison summary with timing and cost data
    """
    # Your implementation here
    pass

# Test your implementation
# Uncomment to test:
# comparison = compare_all_providers("Explain quantum computing in simple terms.")
# print(json.dumps(comparison, indent=2))

---

## Section 12: Summary & Next Steps

### What We Learned Today

✅ **API Setup & Authentication**
- OpenAI, Anthropic, and Google AI clients
- Secure API key management with Colab secrets

✅ **Structured Outputs**
- Pydantic models for type-safe JSON
- Provider-specific approaches:
  - OpenAI: `response_format` with native Pydantic
  - Anthropic: Tool calling pattern
  - Google AI: JSON schema with `response_mime_type`

✅ **Error Handling**
- Common API errors and how to handle them
- Provider-specific exception handling
- Graceful error responses

✅ **Best Practices**
- Token management and cost optimization
- Security checklist
- Provider comparison and selection

### Key Differences Between Providers

| Feature | OpenAI | Anthropic | Google AI |
|---------|--------|-----------|----------|
| **Best For** | General-purpose | Analysis & reasoning | Multimodal, cost-effective |
| **Pricing (per 1M tokens)** | $0.15-$0.60 (mini) | $0.80-$75 (Haiku-Opus) | $0.075-$10 |
| **Context Window** | 128K tokens | 200K tokens | 2M tokens (Gemini 2.5) |
| **Structured Output** | Native Pydantic | Tool calling | JSON schema |
| **Strengths** | Wide adoption | Safety-focused | Long context, multimodal |

### Resources for Further Learning

**Official Documentation:**
- OpenAI: https://platform.openai.com/docs
- Anthropic: https://docs.anthropic.com
- Google AI: https://ai.google.dev/docs

**Community Resources:**
- LangChain: https://python.langchain.com
- Pydantic: https://docs.pydantic.dev

### Preview: Next Session

In the next notebook, we'll cover:

🔮 **Vector Databases & Embeddings**
- What are embeddings and why they matter
- Vector similarity search
- Chunking strategies
- Working with Chroma, Pinecone, FAISS

🔮 **RAG (Retrieval-Augmented Generation)**
- Building a complete RAG pipeline
- Document loading and processing
- Semantic search with citations
- Hands-on: Build a RAG chatbot

---

## Section 13: Bonus - OpenAI Responses API (Advanced)

### What is the Responses API?

OpenAI's **Responses API** provides an alternative API interface. While we've been using the **Chat Completions API** throughout this notebook (which OpenAI will support **indefinitely**), it's valuable to understand when you might choose the Responses API for production applications.

**Key Point:** The Chat Completions API we've used is perfect for this training and will continue to be the industry standard. The Responses API is an advanced option for specific production use cases.

### 🔄 Chat Completions vs Responses API

| Feature | Chat Completions API | Responses API |
|---------|---------------------|---------------|
| **Use Case** | Stateless, single-turn interactions | Stateful, multi-turn conversations |
| **State Management** | Client-side (you manage conversation history) | Server-side (OpenAI manages for you) |
| **Built-in Tools** | No (you implement functions) | Yes (web search, file search, code execution) |
| **Cost** | Standard pricing | 40-80% lower (better cache utilization) |
| **Complexity** | Simple, direct | More features, more complex |
| **Support Status** | Supported **indefinitely** | Current, evolving |
| **Best For** | Training, simple bots, full control | Production apps, complex workflows |
| **GPT-5 Optimization** | Works fine | Optimized for GPT-5 |
| **Learning Curve** | Easier | Steeper |

### 🎯 When to Use Chat Completions (What We've Used)

✅ **Choose Chat Completions API when you need:**
- **Educational/training purposes** - Teaching fundamentals (like this notebook!)
- **Simple, stateless interactions** - One question, one answer
- **Full control over context** - You manage conversation history
- **Lightweight applications** - Chatbots, Q&A systems
- **Industry standard interface** - Most stable, well-documented
- **Custom function calling** - You implement your own tools

### 🚀 When to Use Responses API

✅ **Choose Responses API when you need:**
- **Server-side state management** - Don't want to track conversation history
- **Built-in tools** - Web search, file search, code execution sandbox
- **Cost optimization** - Benefit from 40-80% better caching
- **GPT-5 applications** - GPT-5 works best with Responses API
- **Document analysis** - Legal contracts, research papers, knowledge bases
- **Complex multi-turn workflows** - Extended conversations with context retention

### 📊 Key Advantages of Responses API

1. **Automatic State Management**
   - Pass a `response_id` instead of full conversation history
   - OpenAI maintains conversation context server-side
   - Reduces payload size and complexity

2. **Built-in Tools**
   - Web search without implementing custom functions
   - File search for document analysis
   - Code execution sandbox for running Python code

3. **Better Cache Utilization**
   - 40-80% cost reduction in internal OpenAI tests
   - More efficient prompt caching
   - Reduced latency for repeated patterns

4. **GPT-5 Optimization**
   - GPT-5 orchestrates tools as part of reasoning
   - Legacy endpoints may cause degraded behavior
   - Future models will be optimized for Responses API

### 💡 Example Comparison

**Chat Completions (What We've Used):**
```python
# You manage conversation history
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What is AI?"},
    {"role": "assistant", "content": "AI is..."},
    {"role": "user", "content": "Tell me more"}
]

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages  # Send full history each time
)
```

**Responses API:**
```python
# OpenAI manages state for you
response = client.responses.create(
    model="gpt-4o",
    previous_response_id="resp_abc123",  # Reference to previous state
    messages=[{"role": "user", "content": "Tell me more"}]
)
# Only send new message, not full history
```

### 📚 Migration Timeline

- **Chat Completions API:** Supported **indefinitely** ✅
- **Assistants API:** Being deprecated (migrate by August 26, 2026)
- **Responses API:** Recommended for projects with state/tools

### 🎓 What Should You Use?

**For this training:** We're using **Chat Completions API** because:
- ✅ Perfect for learning fundamentals
- ✅ Simpler, more direct
- ✅ Industry standard that will be supported forever
- ✅ Better for understanding how LLM APIs work

**For production apps:** Consider **Responses API** if:
- You need server-side state management
- You want built-in tools (web search, file search, code execution)
- Cost optimization is critical (40-80% savings)
- You're building with GPT-5

### 🔗 Resources

- [Responses API Documentation](https://platform.openai.com/docs/api-reference/responses)
- [Migration Guide: Chat Completions → Responses](https://platform.openai.com/docs/guides/migrate-to-responses)
- [Responses vs Chat Completions Comparison](https://platform.openai.com/docs/guides/responses-vs-chat-completions)

---

**Bottom Line:** The Chat Completions API you've learned in this notebook is production-ready, will be supported indefinitely, and is perfect for most use cases. The Responses API is an advanced option for specific scenarios requiring server-side state or built-in tools.

---

### 13.1 Hands-On: Basic Responses API

Now let's see the Responses API in action! We'll explore:

1. **Basic text generation** with `client.responses.create()`
2. **Code interpreter tool** for solving math problems

**Key Difference from Chat Completions:**
- Responses API uses `input` instead of `messages`
- Can specify `instructions` for system-level behavior
- Returns `output_text` (simpler than accessing `.choices[0].message.content`)

Let's start with a basic example:

In [ ]:
# Basic Responses API Example
# Compare this to the Chat Completions API we used earlier!

try:
    # Using Responses API (new way)
    response = openai_client.responses.create(
        model="gpt-4o",  # GPT-4o recommended for Responses API
        instructions="You are a helpful coding assistant that explains concepts concisely.",
        input="How do I check if a Python object is an instance of a class?",
    )
    
    # Notice: Much simpler access to output!
    print("📝 Response from Responses API:")
    print("="*60)
    print(response.output_text)
    print("="*60)
    
    # Response metadata
    print(f"\n✅ Model used: {response.model}")
    print(f"✅ Response ID: {response.id}")
    print(f"   (Use this ID with 'previous_response_id' for stateful conversations)")
    
except Exception as e:
    print(f"❌ Error: {str(e)}")
    print("Note: Responses API requires OpenAI Python SDK >= 1.60.0")
    print("Run: !pip install --upgrade openai")

---

### 13.2 Hands-On: Code Interpreter Tool

One of the most powerful features of the Responses API is **built-in tools**. Let's use the **code_interpreter** tool to solve a math problem.

**How Code Interpreter Works:**
1. You provide a problem that requires computation
2. The model writes Python code to solve it
3. Code runs in a **sandboxed container** (secure, isolated environment)
4. Model returns the solution with explanation

**Code Interpreter Configuration:**
```python
tools=[{
    "type": "code_interpreter",
    "container": {"type": "auto"}  # Auto-manages containers
}]
```

**Container Management:**
- `"auto"` mode: Reuses existing container or creates new one
- Containers expire after 20 minutes of inactivity
- Cost: $0.03 per container

Let's solve a math problem:

In [ ]:
# Code Interpreter Tool Example
# The model will write and execute Python code to solve this problem!

try:
    response = openai_client.responses.create(
        model="gpt-4o",
        tools=[
            {
                "type": "code_interpreter",
                "container": {"type": "auto"}  # Auto-manage containers
            }
        ],
        instructions="You are a personal math tutor. Write and run Python code to answer math questions. Show your work step-by-step.",
        input="I need to solve the equation 3x + 11 = 14. Can you help me? Also, verify the answer.",
    )
    
    print("🧮 Math Problem: Solve 3x + 11 = 14")
    print("="*60)
    print("\n📝 Model's Response:")
    print(response.output_text)
    print("="*60)
    
    # Show tool usage information
    print(f"\n✅ Response ID: {response.id}")
    print(f"✅ Model: {response.model}")
    
    # Check if code was executed
    if hasattr(response, 'output') and response.output:
        print(f"✅ Code interpreter was used to solve this problem!")
        print(f"   The model wrote Python code, executed it, and verified the answer.")
    
    print("\n💡 Key Insight:")
    print("   The Responses API automatically ran Python code in a secure sandbox")
    print("   to solve and verify the equation. This is built-in functionality!")
    
except Exception as e:
    print(f"❌ Error: {str(e)}")
    print("\n🔧 Troubleshooting:")
    print("   - Ensure you have OpenAI Python SDK >= 1.60.0")
    print("   - Code interpreter requires GPT-4o or GPT-4.1+ models")
    print("   - Run: !pip install --upgrade openai")


### Questions?

Great work completing this notebook! You now have a solid foundation for working with major LLM APIs.

**Next:** Save this notebook and move on to `02_vector_databases_and_embeddings.ipynb`